## Import

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm
import itertools
import datetime
from datetime import timedelta

## Load data

In [2]:
# 55000
with open(os.path.join('/home/tohone06/','result','paper_drop_srgc_mlpt_202310/train_replaced_with_syn.pkl'), 'rb') as f:
    test = pickle.load(f)

real_sta = pd.concat([test['raw_set'][0],test['test_set'][0]])
real_dyn = test['raw_set'][1] + test['test_set'][1]
# real_sta = test['test_set'][0]
# real_dyn = test['test_set'][1]

syn_sta = test['train_set'][0]
syn_dyn = test['train_set'][1]

In [3]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [4]:
len(syn_sta)

55000

In [5]:
#data logic 55000명
with open(os.path.join('/home/tohone06/RTSGAN-github/drop_ptno_paper_202310_55000.pkl'), 'rb') as f:
    drop_ptno = pickle.load(f)

## dyn_num

In [6]:
#real_dyn
col = real_dyn[0].columns

tot_real = []
for i in tqdm(real_dyn[0].columns):
    var = []
    for k,g in enumerate(real_dyn):
        var.append(g[i].dropna().tolist())
    var = list(itertools.chain(*var))
    tot_real.append([i,var])

100%|██████████| 25/25 [00:44<00:00,  1.78s/it]


In [8]:
#syn_dyn
tot_syn = []
for i in tqdm(syn_dyn[0].columns):
    var = []
    for k,g in enumerate(syn_dyn):
        if k in drop_ptno:
            continue
        var.append(g[i].dropna().tolist())
    var = list(itertools.chain(*var))
    tot_syn.append([i,var])

100%|██████████| 25/25 [02:40<00:00,  6.44s/it]


In [9]:
# syn에서 아예 합성되지 않은 컬럼 추출
not_cols = []
for i in range(len(tot_syn)):
    if len(tot_syn[i][1]) == 0:
        not_cols.append(tot_syn[i][0])

In [10]:
tot_syn = [tot_syn[i] for i in range(len(tot_syn)) if tot_syn[i][0] not in not_cols]

tot_real = [tot_real[i] for i in range(len(tot_real)) if tot_real[i][0] not in not_cols]

In [23]:
numeric = ['CEXM_RSLT_CONT','TIMESTAMP','SGPT_SRMV_LN_CNT','SGPT_MTST_LN_CNT','LINE','CSTR_END_YMD','CSTR_NT', 'RDT_END_YMD']

In [24]:
# num, cat 분리
tot_syn_num = [tot_syn[i] for i in range(len(tot_syn)) if tot_syn[i][0] in numeric]
tot_syn_cat = [tot_syn[i] for i in range(len(tot_syn)) if tot_syn[i][0] not in numeric]

tot_real_num = [tot_real[i] for i in range(len(tot_real)) if tot_real[i][0] in numeric]
tot_real_cat = [tot_real[i] for i in range(len(tot_real)) if tot_real[i][0] not in numeric]

In [25]:
tot_real_num[2][1] = np.array(tot_real_num[2][1]).astype('int')
tot_real_num[3][1] = np.array(tot_real_num[3][1]).astype('int')

In [14]:
#실제 데이터, 합성 데이터, bin_size, column_name
def hd(real,syn,bin_size):
    
    if bin_size < 1:
        real = np.array([round(num,1) for num in real])
        syn = np.array([round(num,1) for num in syn])
        
        syn_min = syn.min()
        syn_max = syn.max()
        real_min = real.min()
        real_max = real.max()

        min_ = np.array([real_min,syn_min]).min()
        max_ = np.array([real_max,syn_max]).max()
        arange = pd.DataFrame(np.arange(min_,max_+0.1,bin_size),columns = ['index'])
        
    
    else: #bin_size = 1
        real = np.floor(real)
        syn = np.floor(syn)
        
        syn_min = syn.min()
        syn_max = syn.max()
        real_min = real.min()
        real_max = real.max()

        min_ = np.array([real_min,syn_min]).min()
        max_ = np.array([real_max,syn_max]).max()

        arange = pd.DataFrame(np.arange(min_,max_+0.1,bin_size),columns = ['index'])
    
    real = pd.DataFrame(real)[0].value_counts().to_frame().reset_index(False).sort_values('index')
    real = pd.merge(arange,real, how = 'left', on = 'index').fillna(0)
    real = np.array(round(real[0]/real[0].sum(),4).tolist())

    syn = pd.DataFrame(syn)[0].value_counts().to_frame().reset_index(False).sort_values('index')
    syn = pd.merge(arange,syn, how = 'left', on = 'index').fillna(0)
    syn = np.array(round(syn[0]/syn[0].sum(),4).tolist())

    bc = sum(np.sqrt(real*syn))
    hd = 1-bc
    
    return hd

In [15]:
#numeric's HD
hd_list = []
bin_list = [1,1,1,1,1,1,1,1]
for k,g in enumerate(zip(tot_real_num,tot_syn_num)):
    hd_list.append([g[0][0],hd(g[0][1],g[1][1],bin_list[k])])

In [16]:
pd.DataFrame(hd_list, columns = ['COL','HD'])

,COL,HD
0,CEXM_RSLT_CONT,0.000437
1,TIMESTAMP,0.023038
2,SGPT_SRMV_LN_CNT,0.058716
3,SGPT_MTST_LN_CNT,0.014385
4,LINE,0.004744
5,CSTR_END_YMD,0.141307
6,CSTR_NT,0.122393
7,RDT_END_YMD,0.089232


## dyn_cat

In [26]:
def hd_cat(real,syn):
    try:
        if sorted(pd.DataFrame(real)[0].unique().tolist()) == sorted(pd.DataFrame(syn)[0].unique().tolist()):
            r = pd.DataFrame(real)[0].value_counts()
            s = pd.DataFrame(syn)[0].value_counts()
            r = r.sort_index()/sum(r)
            s = s.sort_index()/sum(s)
            bc = sum(np.sqrt(r*s))
            hd = round(1-bc,4)
        else: #데이터가 생성이 안되는 문제 등 columns의 unique값이 다른 경우
            r = pd.DataFrame(real)[0].value_counts()
            s = pd.DataFrame(syn)[0].value_counts()
            real_list = r.index.tolist()
            syn_list = s.index.tolist()

            supplement = [c for c in real_list if c not in syn_list]
            for i in range(len(supplement)):
                s.loc[supplement[i]] = 0

            supplement = [c for c in syn_list if c not in real_list]
            for i in range(len(supplement)):
                r.loc[supplement[i]] = 0
            
        r = r.sort_index()/sum(r)
        s = s.sort_index()/sum(s)
        bc = sum(np.sqrt(r*s))
        hd = round(np.sqrt(1-bc),4)
    
    except:
        hd = np.nan
    
    return hd

In [28]:
#stage, m_stage dtype 
tot_syn_cat[9][1] = [str(int(i)) for i in tot_syn_cat[9][1]]
tot_syn_cat[12][1] = [str(int(i)) for i in tot_syn_cat[12][1]]

In [29]:
hd_list = []
for k,g in enumerate(zip(tot_real_cat,tot_syn_cat)):
    hd_list.append([g[0][0],hd_cat(g[0][1],g[1][1])])

In [30]:
pd.DataFrame(hd_list)

,0,1
0,CEXM_RSLT_CONT,0.0196
1,BPTH_BPSY_RSLT_CONT,0.0033
2,IMPT_HM1E_RSLT_CD,0.0000
3,IMPT_HS2E_RSLT_CD,0.0000
4,IMPT_HS6E_RSLT_CD,0.0000
5,IMPT_HP2E_RSLT_CD,0.0000
6,MLPT_KRES_RSLT_CD,0.1010
7,MLPT_NREX_RSLT_CD,0.1193
8,MLPT_BRME_RSLT_CD,0.1060
9,SGPT_PATL_STAG_VL,0.0759


## Static

In [31]:
syn_sta = syn_sta.reset_index(drop = True)
syn_sta = syn_sta.drop(drop_ptno)

In [32]:
#real
real_sta['BSPT_DEAD'] = real_sta['BSPT_DEAD_YMD'].apply(lambda x : '1' if pd.notnull(x) else '0')

syn_sta['BSPT_DEAD'] = syn_sta['BSPT_DEAD_YMD'].apply(lambda x : '1' if pd.notnull(x) else '0')

syn_sta[['BSPT_BRYM','BSPT_FRST_DIAG_YMD','BSPT_DEAD_YMD','seq_len']] = syn_sta[['BSPT_BRYM','BSPT_FRST_DIAG_YMD',
                                                                                 'BSPT_DEAD_YMD','seq_len']].astype('float')


ref_date = pd.to_datetime(20070101, format='%Y%m%d')

real_sta['BSPT_FRST_DIAG_YMD_'] = real_sta['BSPT_FRST_DIAG_YMD'].apply(lambda x: 
                                    datetime.timedelta(x)) + ref_date

syn_sta['BSPT_FRST_DIAG_YMD_'] = syn_sta['BSPT_FRST_DIAG_YMD'].apply(lambda x: 
                                    datetime.timedelta(x)) + ref_date



#생년월일(real)
real_sta['BSPT_BRYM'] = (real_sta['BSPT_BRYM'].apply(lambda x: 
                            datetime.timedelta(x)) + pd.to_datetime('19000101', format = '%Y%m%d'))

#나이(real)
real_sta['BSPT_IDGN_AGE'] = real_sta['BSPT_FRST_DIAG_YMD_'].dt.year - real_sta['BSPT_BRYM'].dt.year

#생년월일(syn)
syn_sta['BSPT_BRYM'] = (syn_sta['BSPT_BRYM'].apply(lambda x: 
                            datetime.timedelta(x)) + pd.to_datetime('19000101', format = '%Y%m%d'))

#나이(syn)
syn_sta['BSPT_IDGN_AGE'] = syn_sta['BSPT_FRST_DIAG_YMD_'].dt.year - syn_sta['BSPT_BRYM'].dt.year




real_sta.drop(columns = ['BSPT_BRYM','BSPT_FRST_DIAG_YMD_','seq_len'],inplace = True)
syn_sta.drop(columns = ['BSPT_BRYM','BSPT_FRST_DIAG_YMD_','seq_len'],inplace = True)


cols = real_sta.columns.tolist()

# real
tot_real_sta = []
for col in cols:
    tot_real_sta.append([col, real_sta[col].dropna().tolist()])

# syn
tot_syn_sta = []
for col in cols:
    tot_syn_sta.append([col, syn_sta[col].dropna().tolist()])

In [33]:
numeric = ['BSPT_IDGN_AGE','BSPT_FRST_DIAG_YMD','BSPT_DEAD_YMD']

# num, cat 분리
sta_syn_num = [tot_syn_sta[i] for i in range(len(tot_syn_sta)) if tot_syn_sta[i][0] in numeric]
sta_syn_cat = [tot_syn_sta[i] for i in range(len(tot_syn_sta)) if tot_syn_sta[i][0] not in numeric]

sta_real_num = [tot_real_sta[i] for i in range(len(tot_real_sta)) if tot_real_sta[i][0] in numeric]
sta_real_cat = [tot_real_sta[i] for i in range(len(tot_real_sta)) if tot_real_sta[i][0] not in numeric]

In [34]:
#numeric's HD
hd_list = []
bin_list = [1,1,1,1]
for k,g in enumerate(zip(sta_real_num,sta_syn_num)):
    hd_list.append([g[0][0],hd(g[0][1],g[1][1],bin_list[k])])

In [35]:
pd.DataFrame(hd_list, columns = ['COL','HD'])

,COL,HD
0,BSPT_FRST_DIAG_YMD,0.135982
1,BSPT_DEAD_YMD,0.246331
2,BSPT_IDGN_AGE,0.007067


In [36]:
#stage, m_stage dtype 
sta_syn_cat[2][1] = [str(int(i)) for i in sta_syn_cat[2][1]]
sta_syn_cat[5][1] = [str(int(i)) for i in sta_syn_cat[5][1]]

In [37]:
#cat's HD
hd_list = []
for k,g in enumerate(zip(sta_real_cat,sta_syn_cat)):
    hd_list.append([g[0][0],hd_cat(g[0][1],g[1][1])])

In [38]:
pd.DataFrame(hd_list)

,0,1
0,BSPT_SEX_CD,0.0059
1,BSPT_FRST_DIAG_CD,0.0348
2,BSPT_STAG_VL,0.0623
3,BSPT_T_STAG_VL,0.0722
4,BSPT_N_STAG_VL,0.0385
5,BSPT_M_STAG_VL,0.0241
6,BSPT_DEAD,0.0214
